In [243]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import pandas as pd

In [227]:
def load_page(path):
    return requests.get(
        "https://www.tudogostoso.com.br" + path, headers={
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:69.0) Gecko/20100101 Firefox/69.0',
            'Accept-Language': 'pt-BR,en-US;q=0.8,es-ES;q=0.6,es;q=0.4,en;q=0.2',
        }, cookies=dict(datadome='EZBFrU0vJxbDSYjlZ1j7Jm_jbviKo8QzFKyEsMytvSJm5-lOx_iiOoOMMgBbjMLQPNtovmxEuOyldOXriCELEPbj6RgBpExmdUBdhUHFZ2'))
    
def find_recipe(name):
    return load_page("/busca?" + urllib.parse.urlencode({"q": name}))

find_recipe("galinha com quiabo")

<Response [200]>

In [74]:
def find_recipe_link(name):
    page = find_recipe(name) 
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup.select('.recipe-card a[href]')[0]['href']

find_recipe_link("galinha com quiabo")

'/receita/136829-galinha-com-quiabo.html'

In [75]:
def find_recipe_details(name):
    print("Fetching " + name)
    link = find_recipe_link(name)
    page = load_page(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    ingredients = [ x.get_text() for x in soup.select('.ingredients-card li') ]
    title = soup.select('.recipe-title h1')[0].get_text().replace("\n", "")
    return {
        'title': title,
        'link': link,
        'ingredients': ingredients
    }
    
find_recipe_details("moqueca de camarão")

Fetching moqueca de camarão


{'title': 'Moqueca de camarão',
 'link': '/receita/9400-moqueca-de-camarao.html',
 'ingredients': ['1/2 kg de camarão médio',
  'Suco de 1 limão',
  '1/2 xícara de café de azeite de dendê',
  '2 cebolas médias cortadas em rodelas',
  '1/2 xícara de chá de coentro',
  '1/2 xícara de chá de salsinha picada',
  '2 cebolinhas verdes cortadas',
  '3 tomates cortados em rodelas',
  'Sal a gosto',
  '1 vidro de leite de coco']}

In [324]:
recipes_names = [
    'galinha com quiabo',
    'strogonoff de frango',
    'strogonoff de carne',
    'escondidinho de carne moida',
    'kibe de feira',
    'esfiha aberta',
    'bolo de cenoura',
    'blooming onion outback',
    'picanha na cerveja preta',
    'pirão de peixe',
    'moqueca de camarão',
    'peixe empanado',
    'peixe no papelote',
    'carbonara',
    'rocambole de carne moída',
    'bifinho de carne moída',
    'sopa de tomate',
    'macarrão ao pesto',
    'macarrão alho e óleo',
    'caldo verde',
    'purê de batatas com alho',
    'batata rosti',
    'mjadra',
    'peixe bella muniere',
    'tomate recheado',
    'sopa de lentilha',
    'arroz carreteiro',
    'feijoada',
    'picadinho de carne',
    'risoto',
    'vinagrete',
    'lentilha com calabresa',
    'feijão tropeiro',
    'farofa',
    'pudim',
    'pastel de carne',
    'cachorro quente',
    'bife acebolado',
    'frango acebolado',
    'torresmo',
    'bolinho de bacalhau',
    'costelinha de porco',
    'filé à milanesa',
    'filé à parmegiana',
    'camarão',
    'acarajé',
    'baião de dois',
    'paella',
    'lasanha',
    'canja de galinha',
    'burrito',
    'guacamole',
    'ceviche',
    'biscoito de polvilho',
    'empadão',
    'sopa de legumes',
    'coxinha',
    'enrroladinho de presunto e queijo',
    'frango xadrez',
    'peixe à belle meunière',
    'salmão'
]

try:
    df = pd.read_csv('recipes.csv', index_col=0)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]   
except:
    df = pd.DataFrame(columns=['title', 'link', 'ingredients'])

try:
    for index, r in enumerate(recipes_names):
        if index >= df.shape[0]:
            df = df.append(find_recipe_details(r), ignore_index=True)
except:
    print("Stopping at", r)
            
df.to_csv('recipes.csv')
df.tail()

Fetching peixe à belle meunière
Fetching salmão


,title,link,ingredients
56,Coxinha prática deliciosa,/receita/1363-coxinha-pratica-deliciosa.html,"['1 xícara (chá) de água', '2 xícaras (chá) de..."
57,Estrogonofe especial de palmito,/receita/18470-estrogonofe-especial-de-palmito...,"['1 vidro de palmito cortado em rodelas', '1 l..."
58,Frango xadrez,/receita/20566-frango-xadrez.html,"['5 filés de peito de frango', '2 colheres (so..."
59,Peixe à Belle Meunière,/receita/36531-peixe-a-belle-meuniere.html,"[500 g de badejo (sirigado) cortado em filés, ..."
60,Salmão ao champignon,/receita/111-salmao-ao-champignon.html,"[2 postas de salmão, sal a gosto, 3 dentes de ..."


In [325]:
import re
import unidecode
import nltk
from nltk.stem.snowball import SnowballStemmer

# nlt.download()
stopwords = nltk.corpus.stopwords.words('portuguese')
# stemmer = SnowballStemmer("portuguese")

df['ingredients'] = df['ingredients'].apply(lambda x: eval(str(x))) 
all_ingredients = unidecode.unidecode(", ".join([ ", ".join(i) for i in df['ingredients']])).lower()

non_ingredients = " ".join(stopwords) + """
    molho litro inteira picada dentes picado creme copo sua peitos cortados preferencia medias cha medio
    pele latas dissolvido cubo ml caldo agua primeira moida como suficiente dar ou para ralado colheres sopa
    sementes refogar picadinha gosto oliva pequena pequeno fritar casca raladas vermelha pitada nao tipo
    xicaras kg use fatiado fatiada cortada cortado copos bem maco fresca amassado caixa fervente
    pacote vermelho grande tempero fina rosca cozido reino fresco extrato rodela virgem descascada
    coxinha asa aproximadamente espremido cubinho soro estepe batido inteiro sobre gelada queijo
"""

all_already_extracted_ingredients = []
try:
    all_already_extracted_ingredients = ", ".join(pd.read_csv('ingredients.csv')['ingredient'])
except:
    print("Nothing to load")

ingredients = []
for _, r in df.iterrows():
    for i in r['ingredients']:
        most_infrequent_word = None
        for w in i.split(" "):
            w = unidecode.unidecode(re.sub("\(|\)|s$", "", w)).lower()
            w = re.sub("oe$", "ao", w)
            w = re.sub("catchup", "ketchup", w)
            w = re.sub("patinho", "carne", w)
            w = re.sub("gema", "ovo", w)
            w = re.sub("clara", "ovo", w)
            w = re.sub("galinha", "frango", w)
            if re.match("^[a-zA-Z]+$", w) is None:
                continue
            if w in non_ingredients:
                continue
            frequency = len(re.findall(w, all_ingredients))
            if frequency < 2:
                continue
            
            # if the ingredient already exist on the list, this counts a lot of points
            existing = len(re.findall(w, all_already_extracted_ingredients))
            frequency -= existing * 2
            
            if most_infrequent_word is None or frequency < most_infrequent_word[0]:
                most_infrequent_word = (frequency, w)
        if most_infrequent_word is None:
            continue
        ingredients.append({ 'ingredient': most_infrequent_word[1], 'title': r['title']})

df_ingredients = pd.DataFrame.from_records(ingredients)
df_ingredients.to_csv('ingredients.csv')
df_ingredients

,ingredient,title
0,tomate,Galinha com quiabo
1,frango,Galinha com quiabo
2,cebola,Galinha com quiabo
3,alho,Galinha com quiabo
4,azeite,Galinha com quiabo
5,quiabo,Galinha com quiabo
6,milho,Galinha com quiabo
7,sal,Galinha com quiabo
8,frango,Strogonoff de frango
9,alho,Strogonoff de frango
